In [6]:
import h5py
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, max_error
import json

# Load the dataset
new_file = h5py.File('data\\ani1x-release-with-nre-and-no-nan.h5', 'r')

def perform_regression_and_correlation(X, y, column_names):
    # Check if there are enough samples for regression
    if len(y) < 2:
        return None, None, None

    # Handle NaN values in input data X
    imputer = SimpleImputer(strategy="mean")
    X = imputer.fit_transform(X)

    #print(X.shape)
    # Split the data into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Linear Regression
    linear_reg = LinearRegression()
    linear_reg.fit(X_train, y_train)
    y_pred_linear = linear_reg.predict(X_test)

    # Ridge Regression
    ridge_reg = Ridge(alpha=0.5)
    ridge_reg.fit(X_train, y_train)
    y_pred_ridge = ridge_reg.predict(X_test)

    # Random Forest Regression
    rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_reg.fit(X_train, y_train)
    y_pred_rf = rf_reg.predict(X_test)

    # Evaluation metrics for linear regression
    r2_linear = r2_score(y_test, y_pred_linear)
    mae_linear = mean_absolute_error(y_test, y_pred_linear)
    max_error_linear = max_error(y_test, y_pred_linear)

    # Evaluation metrics for ridge regression
    r2_ridge = r2_score(y_test, y_pred_ridge)
    mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
    max_error_ridge = max_error(y_test, y_pred_ridge)

    # Evaluation metrics for random forest regression
    r2_rf = r2_score(y_test, y_pred_rf)
    mae_rf = mean_absolute_error(y_test, y_pred_rf)
    max_error_rf = max_error(y_test, y_pred_rf)

    # Calculate the correlation coefficient matrix
    data = np.column_stack((X, y))
    coeff_mat = np.corrcoef(data, rowvar=False)

    regression_results = {
        "Linear Regression": {"R2 Score": r2_linear, "Mean Absolute Error": mae_linear, "Max Absolute Deviation": max_error_linear},
        "Ridge Regression": {"R2 Score": r2_ridge, "Mean Absolute Error": mae_ridge, "Max Absolute Deviation": max_error_ridge},
        "Random Forest Regression": {"R2 Score": r2_rf, "Mean Absolute Error": mae_rf, "Max Absolute Deviation": max_error_rf},
    }

    return regression_results, column_names, coeff_mat

def perform_task(selected_formula):
    # Get the energy datasets
    hf_dz_energies = new_file[selected_formula]['hf_dz.energy'][:]
    hf_tz_energies = new_file[selected_formula]['hf_tz.energy'][:]
    hf_qz_energies = new_file[selected_formula]['hf_qz.energy'][:]
    mp2_dz_energies = new_file[selected_formula]['mp2_dz.corr_energy'][:]
    mp2_tz_energies = new_file[selected_formula]['mp2_tz.corr_energy'][:]
    mp2_qz_energies = new_file[selected_formula]['mp2_qz.corr_energy'][:]
    npno_dz_energies = new_file[selected_formula]['npno_ccsd(t)_dz.corr_energy'][:]
    npno_tz_energies = new_file[selected_formula]['npno_ccsd(t)_tz.corr_energy'][:]
    tnpno_dz_energies = new_file[selected_formula]['tpno_ccsd(t)_dz.corr_energy'][:]
    wb97x_dz_energies = new_file[selected_formula]['wb97x_dz.energy'][:]
    wb97x_tz_energies = new_file[selected_formula]['wb97x_tz.energy'][:]

    # Combine the energies for linear regression
    X_nre = new_file[selected_formula]["nuclear_repulsion_energy"][:]
    X_nre = X_nre.reshape(-1, 1)
    
    X_dft_dz = wb97x_dz_energies
    X_dft_dz = X_dft_dz.reshape(-1, 1)
    
    X_dft_tz = wb97x_tz_energies 
    X_dft_tz = X_dft_tz.reshape(-1, 1)
    
    X_hf_dz = hf_dz_energies
    X_hf_dz = X_hf_dz.reshape(-1, 1)
    
    X_hf_tz = hf_tz_energies 
    X_hf_tz = X_hf_tz.reshape(-1, 1)
    
    X_hf_qz = hf_qz_energies
    X_hf_qz = X_hf_qz.reshape(-1, 1)
    
    X_mp2_dz = mp2_dz_energies
    X_mp2_dz = X_mp2_dz.reshape(-1, 1)
    
    X_mp2_tz = mp2_tz_energies 
    X_mp2_tz = X_mp2_tz.reshape(-1, 1)
    
    X_mp2_qz = mp2_qz_energies
    X_mp2_qz = X_mp2_qz.reshape(-1, 1)

    X_npno_dz = npno_dz_energies 
    X_npno_dz = X_npno_dz.reshape(-1, 1)
    
    X_npno_tz = npno_tz_energies 
    X_npno_tz = X_npno_tz.reshape(-1, 1)
    
    X_tnpno_dz = tnpno_dz_energies 
    X_tnpno_dz = X_tnpno_dz.reshape(-1, 1)
    
    # Get the target values
    ccsd_t_cbs_energies = new_file[selected_formula]["ccsd(t)_cbs.energy"][:]
    y_nre = ccsd_t_cbs_energies
    y_dft_dz = ccsd_t_cbs_energies
    y_hf_dz = ccsd_t_cbs_energies
    y_mp2_dz = ccsd_t_cbs_energies
    y_npno_dz = ccsd_t_cbs_energies
    y_tnpno_dz = ccsd_t_cbs_energies
    y_tz_dz = ccsd_t_cbs_energies
    y_hf_tz = ccsd_t_cbs_energies
    y_mp2_tz = ccsd_t_cbs_energies
    y_npno_tz = ccsd_t_cbs_energies

    # Perform regression for NRE and collect the results
    nre_results, nre_column_names, nre_coeff_mat = perform_regression_and_correlation(X_nre, y_nre, ["NRE", "CCSD"])
    if nre_results is None or nre_coeff_mat is None:
        return

    # Perform regression for DFT_DZ and collect the results
    dft_dz_results, dft_dz_column_names, dft_dz_coeff_mat = perform_regression_and_correlation(X_dft_dz, y_dft_dz, ["DFT_DZ", "CCSD"])
    if dft_dz_results is None or dft_dz_coeff_mat is None:
        return

    # Perform regression for HF_DZ and collect the results
    hf_dz_results, hf_dz_column_names, hf_dz_coeff_mat = perform_regression_and_correlation(X_hf_dz, y_hf_dz, ["HF_DZ", "CCSD"])
    if hf_dz_results is None or hf_dz_coeff_mat is None:
        return

    # Perform regression for MP2_DZ and collect the results
    mp2_dz_results, mp2_dz_column_names, mp2_dz_coeff_mat = perform_regression_and_correlation(X_mp2_dz, y_mp2_dz, ["MP2_DZ", "CCSD"])
    if mp2_dz_results is None or mp2_dz_coeff_mat is None:
        return

    # Perform regression for NPNO_DZ and collect the results
    npno_dz_results, npno_dz_column_names, npno_dz_coeff_mat = perform_regression_and_correlation(X_npno_dz, y_npno_dz, ["NPNO_DZ", "CCSD"])
    if npno_dz_results is None or npno_dz_coeff_mat is None:
        return

    # Perform regression for TNPNO_DZ and collect the results
    tnpno_dz_results, tnpno_dz_column_names, tnpno_dz_coeff_mat = perform_regression_and_correlation(X_tnpno_dz, y_tnpno_dz, ["TNPNO_DZ", "CCSD"])
    if tnpno_dz_results is None or tnpno_dz_coeff_mat is None:
        return

    # Additional regression tasks
    # Using DFT_DZ and HF_DZ to predict CCSD
    X_dft_dz_hf_dz = np.hstack((X_dft_dz, X_hf_dz))
    dz_hf_results, dz_hf_column_names, dz_hf_coeff_mat = perform_regression_and_correlation(X_dft_dz_hf_dz, ccsd_t_cbs_energies, ["DFT_DZ", "HF_DZ", "CCSD"])
    if dz_hf_results is None or dz_hf_coeff_mat is None:
        return

    # Using DFT_DZ and HF_Diff to predict CCSD
    hf_diff_energies = hf_dz_energies - hf_qz_energies
    hf_diff_energies = hf_diff_energies.reshape(-1, 1)  # Reshape to have 2 dimensions
    X_dft_dz_hf_diff = np.hstack(( X_dft_dz, hf_diff_energies))
    dft_diff_results, dft_diff_column_names, dft_diff_coeff_mat = perform_regression_and_correlation(X_dft_dz_hf_diff, y_dft_dz, ["DFT_DZ", "HF_Diff", "CCSD"])
    if dft_diff_results is None or dft_diff_coeff_mat is None:
        return

    # Using DFT_DZ, HF_Diff, NRE to predict CCSD
    hf_diff_energies = hf_dz_energies - hf_qz_energies
    hf_diff_energies = hf_diff_energies.reshape(-1, 1)  # Reshape to have 2 dimensions
    X_nre_reshaped = X_nre[:, 0].reshape(-1, 1)  # Reshape to have 2 dimensions
    X_dft_dz_hf_nre = np.hstack((X_dft_dz, hf_diff_energies, X_nre_reshaped))
    dft_diff_nre_results, dft_diff_nre_column_names, dft_diff_nre_coeff_mat = perform_regression_and_correlation(X_dft_dz_hf_nre, y_dft_dz, ["DFT_DZ", "HF_Diff", "NRE", "CCSD"])

    if dft_diff_nre_results is None or dft_diff_nre_coeff_mat is None:
        return
    
    # Using DFT_DZ and DFT_TZ to predict CCSD
    X_dft_dz_tz = np.hstack((X_dft_dz, X_dft_tz))
    dft_dz_tz_results, dft_dz_tz_column_names, dft_dz_tz_coeff_mat = perform_regression_and_correlation(X_dft_dz_tz, ccsd_t_cbs_energies, ["DFT_DZ", "DFT_TZ", "CCSD"])
    if dft_dz_tz_results is None or dft_dz_tz_coeff_mat is None:
        return

    # Using HF_DZ, HF_TZ, and HF_QZ to predict CCSD
    X_hf_dz_tz_qz = np.hstack((X_hf_dz, X_hf_tz, X_hf_qz))
    hf_dz_tz_qz_results, hf_dz_tz_qz_column_names, hf_dz_tz_qz_coeff_mat = perform_regression_and_correlation(X_hf_dz_tz_qz, ccsd_t_cbs_energies, ["HF_DZ", "HF_TZ", "HF_QZ", "CCSD"])
    if hf_dz_tz_qz_results is None or hf_dz_tz_qz_coeff_mat is None:
        return

    # Using MP2_DZ, MP2_TZ, and MP2_QZ to predict CCSD
    X_mp2_dz_tz_qz = np.hstack((X_mp2_dz, X_mp2_tz, X_mp2_qz))
    mp2_dz_tz_qz_results, mp2_dz_tz_qz_column_names, mp2_dz_tz_qz_coeff_mat = perform_regression_and_correlation(X_mp2_dz_tz_qz, ccsd_t_cbs_energies, ["MP2_DZ", "MP2_TZ", "MP2_QZ", "CCSD"])
    if mp2_dz_tz_qz_results is None or mp2_dz_tz_qz_coeff_mat is None:
        return

    # Using NPNO_DZ and NPNO_TZ to predict CCSD
    X_npno_dz_tz = np.hstack((X_npno_dz, X_npno_tz))
    npno_dz_tz_results, npno_dz_tz_column_names, npno_dz_tz_coeff_mat = perform_regression_and_correlation(X_npno_dz_tz, ccsd_t_cbs_energies, ["NPNO_DZ", "NPNO_TZ", "CCSD"])
    if npno_dz_tz_results is None or npno_dz_tz_coeff_mat is None:
        return

    # Store the results and coefficients in a dictionary
    results_dict = {
        "NRE": {"Regression Results": nre_results, "Column Names": nre_column_names, "Coefficients": nre_coeff_mat.tolist()},
        "DFT_DZ": {"Regression Results": dft_dz_results, "Column Names": dft_dz_column_names, "Coefficients": dft_dz_coeff_mat.tolist()},
        "HF_DZ": {"Regression Results": hf_dz_results, "Column Names": hf_dz_column_names, "Coefficients": hf_dz_coeff_mat.tolist()},
        "MP2_DZ": {"Regression Results": mp2_dz_results, "Column Names": mp2_dz_column_names, "Coefficients": mp2_dz_coeff_mat.tolist()},
        "NPNO_DZ": {"Regression Results": npno_dz_results, "Column Names": npno_dz_column_names, "Coefficients": npno_dz_coeff_mat.tolist()},
        "TNPNO_DZ": {"Regression Results": tnpno_dz_results, "Column Names": tnpno_dz_column_names, "Coefficients": tnpno_dz_coeff_mat.tolist()},
        "DFT_DZ_HF_DZ": {"Regression Results": dz_hf_results, "Column Names": dz_hf_column_names, "Coefficients": dz_hf_coeff_mat.tolist()},
        "DFT_DZ_HF_Diff": {"Regression Results": dft_diff_results, "Column Names": dft_diff_column_names, "Coefficients": dft_diff_coeff_mat.tolist()},
        "DFT_DZ_HF_NRE": {"Regression Results": dft_diff_nre_results, "Column Names": dft_diff_nre_column_names, "Coefficients": dft_diff_nre_coeff_mat.tolist()},
        "DFT_DZ_DFT_TZ": {"Regression Results": dft_dz_tz_results, "Column Names": dft_dz_tz_column_names, "Coefficients": dft_dz_tz_coeff_mat.tolist()},
        "HF_DZ_HF_TZ_HF_QZ": {"Regression Results": hf_dz_tz_qz_results, "Column Names": hf_dz_tz_qz_column_names, "Coefficients": hf_dz_tz_qz_coeff_mat.tolist()},
        "MP2_DZ_MP2_TZ_MP2_QZ": {"Regression Results": mp2_dz_tz_qz_results, "Column Names": mp2_dz_tz_qz_column_names, "Coefficients": mp2_dz_tz_qz_coeff_mat.tolist()},
        "NPNO_DZ_NPNO_TZ": {"Regression Results": npno_dz_tz_results, "Column Names": npno_dz_tz_column_names, "Coefficients": npno_dz_tz_coeff_mat.tolist()},
    }

    return results_dict

# Loop through each molecular formula in the dataset
molecular_formulas = list(new_file.keys())
results_json = {}
for molecule in molecular_formulas:
    results_json[molecule] = perform_task(molecule)

# Writing data to a JSON file
with open("energy_data_error_metrics_and_correlations1.json", "w") as json_file:
    json.dump(results_json, json_file, indent=4)
print("Energy data with metrics and correlation coefficients has been written to 'energy_data_error_metrics_and_correlations1.json' file.")

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

...

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\amahi\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Energy data with metrics and correlation coefficients has been written to 'energy_data_error_metrics_and_correlations1.json' file.


In [8]:
import json

# Read the JSON file
with open("energy_data_error_metrics_and_correlations1.json", "r") as json_file:
    data = json.load(json_file)

# Display a subset of the JSON data
subset_data = {}
start_index = 0
end_index = 5  # Change this value to the ending index you want

# Select the subset of data based on the specified start and end index
keys = list(data.keys())[start_index:end_index]

# Create a new dictionary with the selected keys and corresponding values
for key in keys:
    subset_data[key] = data[key]

# Pretty print the subset of the JSON data
print(json.dumps(subset_data, indent=4))


{
    "C10H10": {
        "NRE": {
            "Regression Results": {
                "Linear Regression": {
                    "R2 Score": 0.00868252294858729,
                    "Mean Absolute Error": 0.08983512384486972,
                    "Max Absolute Deviation": 0.1467163697176943
                },
                "Ridge Regression": {
                    "R2 Score": 0.008684220461755432,
                    "Mean Absolute Error": 0.08983506201071237,
                    "Max Absolute Deviation": 0.14671637153145412
                },
                "Random Forest Regression": {
                    "R2 Score": 0.7451951793806646,
                    "Mean Absolute Error": 0.037949118191431805,
                    "Max Absolute Deviation": 0.10393470177439212
                }
            },
            "Column Names": [
                "NRE",
                "CCSD"
            ],
            "Coefficients": [
                [
                    1.0,
                    -0